In [1]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_csv("D:/Kuliah/NonAkademik/FindIT 2024/dataset/train_features.csv")
df1.info()

: 

Merubah data tahun kelahiran menjadi umur

In [ ]:
df2=pd.read_csv('D:/Kuliah/NonAkademik/FindIT 2024/dataset/train_labels.csv')
df2

: 

In [ ]:
df=pd.concat([df1,df2],axis=1)
df

: 

In [ ]:
df.dropna()

: 

In [ ]:
df.columns

: 

In [ ]:
import numpy as np

df['umur'] = 2015 - df['tahun_kelahiran']
df = df.drop("tahun_kelahiran", axis=1)
# print(df["umur"].value_counts())

Q1 = df['umur'].quantile(0.25)
Q3 = df['umur'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 2 * IQR
upper_bound = Q3 + 2 * IQR

outliers = df[(df['umur'] < lower_bound) | (df['umur'] > upper_bound)]

df.loc[(df['umur'] < lower_bound) | (df['umur'] > upper_bound), 'umur'] = np.nan

df.head()


: 

Perbaikan Data Pendidikan

In [ ]:
df['pendidikan'] = df['pendidikan'].replace(5, np.nan)


: 

Perbaikan Data Pernikahan

In [ ]:
df['status_pernikahan'] = df['status_pernikahan'].replace(5, np.nan)

: 

Merubah Data tanggal menjadi anggota

In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset

df['tanggal_menjadi_anggota'] = pd.to_datetime(df['tanggal_menjadi_anggota'], errors='coerce')

current_date = pd.to_datetime('2014-07-01')

df['keanggotaan'] = ((current_date.year - df['tanggal_menjadi_anggota'].dt.year) * 12 +
                            (current_date.month - df['tanggal_menjadi_anggota'].dt.month))

df = df.drop("tanggal_menjadi_anggota", axis=1)



: 

In [ ]:
df['keanggotaan'].describe()

: 

Data Anak

In [ ]:
# df["jumlah_anak"] = df['jumlah_anak_balita'] + df['jumlah_anak_remaja']
# df['jumlah_anak'].value_counts()

: 

Pembuangan Data

In [ ]:
# drop = ['jumlah_anak_balita', 'jumlah_anak_remaja']
# df = df.drop(drop, axis=1)

: 

Pre pro tahap 2

Mapping Data Pendidikan

In [ ]:
df['pendidikan'] = df['pendidikan'].replace({'Magister': 'Pasca_Sarjana', 'Doktor': 'Pasca_Sarjana'})
# df['pendidikan'] = df['pendidikan'].replace({'SMP': 'Sekolah_Menengah', 'SMA': 'Sekolah_Menengah'})

# # 0.75
# education_order = {
#     'SMP': 0,
#     'SMA': 1,        
#     'Sarjana': 2,   
#     'Magister': 3,
#     'Doktor': 4
# }

# # 0.7612
# education_order = {
#     'Sekolah_Menengah': 0,       
#     'Sarjana': 1,   
#     'Magister': 2,
#     'Doktor': 3
# }

# 0.7621 (Terbaik)
education_order = {
    'SMP': 0,
    'SMA': 1,        
    'Sarjana': 2,   
    'Pasca_Sarjana': 3
}

# # 0.7590
# education_order = {
#     'Sekolah_Menengah': 0,     
#     'Sarjana': 1,   
#     'Pasca_Sarjana': 2
# }

df['pendidikan'] = df['pendidikan'].map(education_order)



: 

Mapping Status Pernikahan

In [ ]:
import pandas as pd
# df['status_pernikahan'] = df['status_pernikahan'].replace({'Sendiri': 'Single', 'Cerai':'Single', 'Cerai Mati': 'Single'})
# df['status_pernikahan'] = df['status_pernikahan'].replace({'Cerai':'Pisah', 'Cerai Mati': 'Pisah'})

# 0.7621
status_order = {
    'Sendiri': 0,
    'Rencana Menikah': 1,  # Planning to marry
    'Menikah': 2,
    'Cerai': 3,
    'Cerai Mati': 4
}

# # 0.7519
# status_order = {
#     'Sendiri': 0,
#     'Rencana Menikah': 1,  # Planning to marry
#     'Menikah': 2,
#     'Pisah': 3
# }

# # 0.7432
# status_order = {
#     'Single': 0,
#     'Rencana Menikah': 1,  # Planning to marry
#     'Menikah': 2
# }


df['status_pernikahan'] = df['status_pernikahan'].map(status_order)

# print(df['status_pernikahan'].value_counts())
# df.head()

: 

In [ ]:
print(df['keluhan'].value_counts())

: 

In [ ]:
df.columns

: 

In [ ]:
df.isna().sum()

: 

Imputasi Kategori

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd

def impute_categorical_data_with_lightgbm(df, column_to_impute, categorical_columns):
    df_with_missing = df[df[column_to_impute].isna()].copy()
    df_without_missing = df[~df[column_to_impute].isna()].copy()

    X = df_without_missing.drop(columns=[column_to_impute, 'jumlah_promosi'])
    y = df_without_missing[column_to_impute]

    for col in categorical_columns:
        if col != column_to_impute and col in X.columns:
            X[col] = X[col].astype('category')
        if col in df_with_missing.columns:
            df_with_missing[col] = df_with_missing[col].astype('category')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    model = lgb.LGBMClassifier(
        n_estimators=100,
        learning_rate=0.1,
        num_leaves=31,
        random_state=0,
        class_weight='balanced', 
        verbose=-1
    )
    categorical_features_indices = [X_train.columns.get_loc(c) for c in categorical_columns if c in X_train]
    model.fit(X_train, y_train, categorical_feature=categorical_features_indices)

    y_pred = model.predict(X_test)
    f1_metric = f1_score(y_test, y_pred, average='macro')

    imputed_values = model.predict(df_with_missing.drop(columns=[column_to_impute, 'jumlah_promosi']))
    df.loc[df_with_missing.index, column_to_impute] = imputed_values

    return df, f1_metric

categorical_columns = ['status_pernikahan', 'pendidikan', 'keluhan']
imputation_results = {}
for column in categorical_columns:
    if column in df.columns:
        df, f1_metric = impute_categorical_data_with_lightgbm(df, column, categorical_columns)
        imputation_results[column] = f1_metric
    else:
        imputation_results[column] = None

print(imputation_results)
df.head()

: 

{'status_pernikahan': 0.41326530753954477, 'pendidikan': 0.6350493451558283, 'keluhan': 0.7493084370677732}

Imputasi Numerik

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np

def impute_numerical_data_with_lightgbm(df, column_to_impute, categorical_columns):
    if 'jumlah_promosi' in df.columns:
        jumlah_promosi = df['jumlah_promosi'].copy()
        df = df.drop(columns=['jumlah_promosi'])
    else:
        jumlah_promosi = None
    
    df_with_missing = df[df[column_to_impute].isna()].copy()
    df_without_missing = df[~df[column_to_impute].isna()].copy()

    X = df_without_missing.drop(columns=[column_to_impute])
    y = df_without_missing[column_to_impute]

    for col in categorical_columns:
        if col in X.columns:
            X[col] = X[col].astype('category')
        if col in df_with_missing.columns:
            df_with_missing.loc[:, col] = df_with_missing[col].astype('category')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    model = lgb.LGBMRegressor(
        n_estimators=100,
        learning_rate=0.1,
        num_leaves=31,
        random_state=0
    )
    model.fit(X_train, y_train, categorical_feature=[col for col in categorical_columns if col in X.columns])

    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)

    imputed_values = model.predict(df_with_missing.drop(columns=[column_to_impute]))
    df.loc[df_with_missing.index, column_to_impute] = np.round(imputed_values).astype(int)

    if jumlah_promosi is not None:
        df['jumlah_promosi'] = jumlah_promosi

    return df, mae


categorical_columns = ['status_pernikahan', 'pendidikan', 'keluhan']

integer_columns = [
    'pendapatan', 'jumlah_anak_balita', 'jumlah_anak_remaja', 'terakhir_belanja', 'belanja_buah', 'belanja_daging', 
    'belanja_ikan', 'belanja_kue', 'pembelian_diskon', 'pembelian_web', 'pembelian_toko', 
    'umur', 'keanggotaan'
]

imputation_results = {}
for column in integer_columns:
    if column in df.columns:
        df, mae = impute_numerical_data_with_lightgbm(df, column, categorical_columns)
        imputation_results[column] = mae
    else:
        print(f"The column '{column}' was not found in the DataFrame.")
        imputation_results[column] = None

print(imputation_results)
df.head()


: 

Memastikan data tidak negatif

In [ ]:
columns_to_correct = ['belanja_kue', 'belanja_daging', 'belanja_ikan', 'belanja_buah']

for column in columns_to_correct:
    df[column] = df[column].apply(lambda x: max(x, 0))

: 

In [ ]:
# print(df['keluhan'].value_counts())

: 

Drop Data

In [ ]:
drop = ['keanggotaan']
df = df.drop(drop, axis=1)

: 

In [ ]:
drop = ['keluhan']
df = df.drop(drop, axis=1)

: 

Pre pro tahap 2 (Generate New Variables)

In [ ]:
print(df.columns)

: 

Daftar Variabel Baru

In [ ]:
from sklearn.preprocessing import QuantileTransformer
from scipy.stats.mstats import winsorize

# # Variabel Pembantu
df['jumlah_pembelian'] = df['pembelian_diskon'] + df['pembelian_web'] + df['pembelian_toko']
df['total_belanja'] = df['belanja_daging'] + df['belanja_ikan'] + df['belanja_buah'] + df['belanja_kue']
df["jumlah_anak"] = df['jumlah_anak_balita'] + df['jumlah_anak_remaja']

# 8018
df['winsorized_pendapatan'] = winsorize(df['pendapatan'], limits=[0.01, 0.01])
# 8038
df['log_pendapatan'] = np.log(df['pendapatan'] + 1)
# 8097
df['ranked_pendapatan'] = df['pendapatan'].rank(method='average')
# 8101
df['cbrt_pendapatan'] = np.cbrt(df['pendapatan'])
# 8108
df['rasio_pembelian_diskon'] = df['pembelian_diskon']/(df['jumlah_pembelian']+1)
# 8115
df['proporsi_kue'] = df['belanja_kue'] / (df['belanja_buah'] + df['belanja_daging'] + df['belanja_ikan'] + df['belanja_kue'] + 1)
# 8165
df['pendapatan_kelompok_umur'] = df['winsorized_pendapatan'] / df['umur']
# 8161
df['log_shopping_frequency'] = np.log(df['total_belanja']+1)
# 8140
df['proporsi_buah'] = df['belanja_buah'] / (df['belanja_buah'] + df['belanja_daging'] + df['belanja_ikan'] + df['belanja_kue'] + 1)
# 8181
df['rasio_belanja_ikan'] = df['belanja_ikan'] / df['pendapatan']
# 8181
df['rasio_belanja_kue'] = df['belanja_kue'] / df['pendapatan']
# 8212
df['total_pembelian_toko_web_pendapatan'] = (df['pembelian_toko'] + df['pembelian_web']) / (df['pendapatan'] + 1)
# 8199
df['pendapatan_adjusted_umur_pendidikan'] = df['pendapatan'] / (df['umur'] * (df['pendidikan'].astype(int) + 1))
# 8205
df['rasio_pembelian_toko'] = df['pembelian_toko']/(df['jumlah_pembelian']+1)
# 8214
df['income_stability_score'] = df['pendapatan'] / (df[['belanja_buah', 'belanja_daging', 'belanja_ikan', 'belanja_kue']].std(axis=1) + 1)
# 8204
mean_income = np.log(df['pendapatan'].mean() + 1)
df['log_pendapatan_vs_mean'] = np.log(df['pendapatan'] + 1) - mean_income
# 8232
df['log_age'] = np.log((df['umur']))


# # 8212. 8184
# df['budget_belanja_kue_perhari'] = (df['total_belanja'] - df['belanja_kue']) / (df['terakhir_belanja'] + 1) 
# 8157
# df['high_value_customer'] = ((df['winsorized_pendapatan'] > df['winsorized_pendapatan'].quantile(0.75)) & (df['terakhir_belanja'] < df['terakhir_belanja'].quantile(0.25))).astype(int)
# 8102. 8133
# df['log_spending_ratio_kue'] = np.log(df['belanja_kue'] / (df['pendapatan'] + 1) + 1)

# 8106
# df['log_spending_ratio_ikan'] = np.log(df['belanja_ikan'] / (df['pendapatan'] + 1) + 1)
# 8156
# df['log_spending_ratio_buah'] = np.log(df['belanja_buah'] / (df['pendapatan'] + 1) + 1)
# 8159
# df['recency_frequency_score'] = (df['terakhir_belanja'] + 1) * df['pembelian_diskon']

# # 8127
# df['rasio_belanja_daging'] = df['belanja_daging'] / df['pendapatan']
# 8105
# df['pendapatan_per_jumlah_anak'] = df['pendapatan'] / (df['jumlah_anak'] + 1)
# 8130
# df['interaksi_pendidikan_pendapatan'] = (df['pendidikan'].astype(int) + 1) * df['winsorized_pendapatan']
# 8180
# df['rasio_belanja_buah'] = df['belanja_buah'] / df['winsorized_pendapatan'] 
# 8170
# df['pendapatan_terhadap_buah_kue'] = df['winsorized_pendapatan'] / (df['belanja_buah'] + df['belanja_kue'] + 1)
# 8075. 8135
# df['total_belanja_protein'] = df['belanja_daging'] + df['belanja_ikan'] + df['belanja_buah'] 
# buah=8170, daging=8127
# df['rasio_belanja_buah'] = df['belanja_daging'] / df['pendapatan']
# 8126
# df['rasio_pembelian_web'] = df['pembelian_web']/(df['jumlah_pembelian']+1)
# 8141
# df['log_dependency_ratio'] = np.log(df['pendapatan'] / (df['jumlah_anak_balita'] + df['jumlah_anak_remaja'] + 1) + 1)
# 8148
# df['log_spending_ratio_daging'] = np.log(df['belanja_daging'] / (df['winsorized_pendapatan'] + 1) + 1)
# 8193
# df['log_terakhir_belanja'] = np.log(df['terakhir_belanja'] + 1)




: 

In [ ]:
drop = ['jumlah_pembelian', 'total_belanja', 'jumlah_anak']
df = df.drop(drop, axis=1)

: 

In [ ]:
# df.info()

: 

In [ ]:
df.to_csv("DataTrain10.csv", index=False)
# df.to_excel("DataTrain6.xlsx", index=False)

: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('D:/Kuliah/NonAkademik/FindIT 2024/DataTrain10.csv')

if 'jumlah_promosi' not in df.columns:
    raise ValueError("The target variable 'jumlah_promosi' is not in the DataFrame.")

X = df.drop(columns=['jumlah_promosi'])
y = df['jumlah_promosi']

model = ExtraTreesClassifier(
    n_estimators=100,
    random_state=0,
    criterion='gini',
    max_depth=None,
    class_weight='balanced'  
)

smote = SMOTE(random_state=0)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
f1_scores = []

for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    model.fit(X_train_smote, y_train_smote)

    y_pred = model.predict(X_test)

    score = f1_score(y_test, y_pred, average='macro')
    f1_scores.append(score)

average_f1_score = np.mean(f1_scores)
print(f"Average macro F1 Score: {average_f1_score:.4f}")

perm_importance = permutation_importance(model, X, y, n_repeats=10, random_state=0)

sorted_idx = perm_importance.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(perm_importance.importances[sorted_idx].T, vert=False, labels=X.columns[sorted_idx])
ax.set_title("Permutation Importances")
plt.show()

: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('D:/Kuliah/NonAkademik/FindIT 2024/DataTrain10.csv')

if 'jumlah_promosi' not in df.columns:
    raise ValueError("The target variable 'jumlah_promosi' is not in the DataFrame.")

X = df.drop(columns=['jumlah_promosi'])
y = df['jumlah_promosi']

model = ExtraTreesClassifier(
    n_estimators=393,
    max_depth= 42,
    min_samples_split=3,
    min_samples_leaf=1,
    max_features='log2',
    random_state=0,
    criterion='gini',
    class_weight='balanced'  
)
# {'n_estimators': 399, 'max_depth': 91, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'log2'}
# {'n_estimators': 393, 'max_depth': 42, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'log2'}

smote = SMOTE(random_state=0)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
f1_scores = []

for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    model.fit(X_train_smote, y_train_smote)

    y_pred = model.predict(X_test)

    score = f1_score(y_test, y_pred, average='macro')
    f1_scores.append(score)

average_f1_score = np.mean(f1_scores)
print(f"Average macro F1 Score: {average_f1_score:.4f}")

# perm_importance = permutation_importance(model, X, y, n_repeats=10, random_state=0)

# sorted_idx = perm_importance.importances_mean.argsort()

# fig, ax = plt.subplots()
# ax.boxplot(perm_importance.importances[sorted_idx].T, vert=False, labels=X.columns[sorted_idx])
# ax.set_title("Permutation Importances")
# plt.show()


: 

Tunning

In [ ]:
# import pandas as pd
# import numpy as np
# import optuna
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import f1_score, make_scorer
# from imblearn.over_sampling import SMOTE
# import matplotlib.pyplot as plt
# import seaborn as sns

# df = pd.read_csv('D:/Kuliah/NonAkademik/FindIT 2024/DataTrain10.csv')

# if 'jumlah_promosi' not in df.columns:
#     raise ValueError("The target variable 'jumlah_promosi' is not in the DataFrame.")

# X = df.drop(columns=['jumlah_promosi'])
# y = df['jumlah_promosi']

# def objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 50, 300)
#     max_depth = trial.suggest_int('max_depth', 5, 50)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
#     max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    
#     model = ExtraTreesClassifier(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         min_samples_split=min_samples_split,
#         min_samples_leaf=min_samples_leaf,
#         max_features=max_features,
#         random_state=0,
#         criterion='gini',
#         class_weight='balanced'  
#     )

#     kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
#     f1_scores = []

#     for train_index, test_index in kfold.split(X, y):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y[train_index], y[test_index]

#         smote = SMOTE(random_state=0)
#         X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

#         model.fit(X_train_smote, y_train_smote)

#         y_pred = model.predict(X_test)

#         score = f1_score(y_test, y_pred, average='macro')
#         f1_scores.append(score)

#     average_f1_score = np.mean(f1_scores)
#     return average_f1_score

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=200)

# print("Best hyperparameters:", study.best_params)

# best_model = ExtraTreesClassifier(
#     n_estimators=study.best_params['n_estimators'],
#     max_depth=study.best_params['max_depth'],
#     min_samples_split=study.best_params['min_samples_split'],
#     min_samples_leaf=study.best_params['min_samples_leaf'],
#     max_features=study.best_params['max_features'],
#     random_state=0,
#     criterion='gini',
#     class_weight='balanced'
# )


: 

In [ ]:
df.info()

: 

In [ ]:
import sklearn
print("Scikit-Learn version:", sklearn.__version__)


: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import SMOTE
import joblib
import pickle
import gzip

df = pd.read_csv('D:/Kuliah/NonAkademik/FindIT 2024/DataTrain10.csv')

if 'jumlah_promosi' not in df.columns:
    raise ValueError("The target variable 'jumlah_promosi' is not in the DataFrame.")

X = df.drop(columns=['jumlah_promosi'])
y = df['jumlah_promosi']

# model = ExtraTreesClassifier(
#     n_estimators=100,
#     random_state=0,
#     criterion='gini',  
#     max_depth=None,  
#     class_weight='balanced'  
# )

model = ExtraTreesClassifier(
    n_estimators=393,
    max_depth= 42,
    min_samples_split=3,
    min_samples_leaf=1,
    max_features='log2',
    random_state=0,
    criterion='gini',
    class_weight='balanced'  # To handle class imbalance
)
# # {'n_estimators': 393, 'max_depth': 42, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'log2'}


smote = SMOTE(random_state=0)
X_smote, y_smote = smote.fit_resample(X, y)

model.fit(X_smote, y_smote)

with gzip.open('model.pkl.gz', 'wb') as f:
  pickle.dump(model, f)

test_df = pd.read_csv(r"D:/Kuliah/NonAkademik/FindIT 2024/DataTest.csv")
submission_df = pd.read_csv(r"D:/Kuliah/NonAkademik/FindIT 2024/dataset/Submission.csv")

X_test = test_df  

predictions = model.predict(X_test).astype(int)

result_df = pd.DataFrame({
    'ID': submission_df['ID'],  
    'jumlah_promosi': predictions
})

print(predictions)
result_df.to_csv(r"D:/Kuliah/NonAkademik/FindIT 2024/predictions10.csv", index=False)

: 

In [ ]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
from scipy.stats.mstats import winsorize

# Load model yang telah dilatih
with open('D:/Kuliah/NonAkademik/FindIT 2024/model.pkl', 'rb') as f:
    model = pickle.load(f)
    
data = {
    'tahun_kelahiran': [1968],
    'pendidikan': ['SMA'],
    'status_pernikahan': ['Menikah'],
    'pendapatan': [29857000],
    'jumlah_anak_balita': [0],
    'jumlah_anak_remaja': [0],
    'terakhir_belanja': [34],
    'belanja_buah': [8092.0],
    'belanja_daging': [22253.0],
    'belanja_ikan': [30345.0],
    'belanja_kue': [26299],
    'pembelian_diskon': [2],
    'pembelian_web': [0],
    'pembelian_toko': [5],
    'keluhan': [0],
    'tanggal_menjadi_anggota': ['2013-08-06']
}

df = pd.DataFrame(data)


df['umur'] = 2015 - df['tahun_kelahiran']
df = df.drop("tahun_kelahiran", axis=1)
# print(df["umur"].value_counts())

Q1 = df['umur'].quantile(0.25)
Q3 = df['umur'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 2 * IQR
upper_bound = Q3 + 2 * IQR

outliers = df[(df['umur'] < lower_bound) | (df['umur'] > upper_bound)]

df.loc[(df['umur'] < lower_bound) | (df['umur'] > upper_bound), 'umur'] = np.nan


df['tanggal_menjadi_anggota'] = pd.to_datetime(df['tanggal_menjadi_anggota'], errors='coerce')

current_date = pd.to_datetime('2014-07-01')

df['keanggotaan'] = ((current_date.year - df['tanggal_menjadi_anggota'].dt.year) * 12 +
                            (current_date.month - df['tanggal_menjadi_anggota'].dt.month))

df = df.drop("tanggal_menjadi_anggota", axis=1)
df['pendidikan'] = df['pendidikan'].replace({'Magister': 'Pasca_Sarjana', 'Doktor': 'Pasca_Sarjana'})

education_order = {
    'SMP': 0, 'SMA': 1, 'Sarjana': 2, 'Pasca Sarjana': 3
}

status_order = {
    'Sendiri': 0, 'Rencana Menikah': 1, 'Menikah': 2, 'Cerai': 3, 'Cerai Mati': 4
}
df['pendidikan'] = df['pendidikan'].map(education_order)
df['status_pernikahan'] = df['status_pernikahan'].map(status_order)
columns_to_correct = ['belanja_kue', 'belanja_daging', 'belanja_ikan', 'belanja_buah']

for column in columns_to_correct:
    df[column] = df[column].apply(lambda x: max(x, 0))
drop = ['keanggotaan']
df = df.drop(drop, axis=1)
drop = ['keluhan']
df = df.drop(drop, axis=1)
df['jumlah_pembelian'] = df['pembelian_diskon'] + df['pembelian_web'] + df['pembelian_toko']
df['total_belanja'] = df['belanja_daging'] + df['belanja_ikan'] + df['belanja_buah'] + df['belanja_kue']
df["jumlah_anak"] = df['jumlah_anak_balita'] + df['jumlah_anak_remaja']

# 8018
df['winsorized_pendapatan'] = winsorize(df['pendapatan'], limits=[0.01, 0.01])
# 8038
df['log_pendapatan'] = np.log(df['pendapatan'] + 1)
# 8097
df['ranked_pendapatan'] = df['pendapatan'].rank(method='average')
# 8101
df['cbrt_pendapatan'] = np.cbrt(df['pendapatan'])
# 8108
df['rasio_pembelian_diskon'] = df['pembelian_diskon']/(df['jumlah_pembelian']+1)
# 8115
df['proporsi_kue'] = df['belanja_kue'] / (df['belanja_buah'] + df['belanja_daging'] + df['belanja_ikan'] + df['belanja_kue'] + 1)
# 8165
df['pendapatan_kelompok_umur'] = df['winsorized_pendapatan'] / df['umur']
# 8161
df['log_shopping_frequency'] = np.log(df['total_belanja']+1)
# 8140
df['proporsi_buah'] = df['belanja_buah'] / (df['belanja_buah'] + df['belanja_daging'] + df['belanja_ikan'] + df['belanja_kue'] + 1)
# 8181
df['rasio_belanja_ikan'] = df['belanja_ikan'] / df['pendapatan']
# 8181
df['rasio_belanja_kue'] = df['belanja_kue'] / df['pendapatan']
# 8212
df['total_pembelian_toko_web_pendapatan'] = (df['pembelian_toko'] + df['pembelian_web']) / (df['pendapatan'] + 1)
# 8199
df['pendapatan_adjusted_umur_pendidikan'] = df['pendapatan'] / (df['umur'] * (df['pendidikan'].astype(int) + 1))
# 8205
df['rasio_pembelian_toko'] = df['pembelian_toko']/(df['jumlah_pembelian']+1)
# 8214
df['income_stability_score'] = df['pendapatan'] / (df[['belanja_buah', 'belanja_daging', 'belanja_ikan', 'belanja_kue']].std(axis=1) + 1)
# 8204
mean_income = np.log(df['pendapatan'].mean() + 1)
df['log_pendapatan_vs_mean'] = np.log(df['pendapatan'] + 1) - mean_income
# 8232
df['log_age'] = np.log((df['umur']))
drop = ['jumlah_pembelian', 'total_belanja', 'jumlah_anak']
df = df.drop(drop, axis=1)
# df.drop('ID', axis=1, inplace=True)


# Melakukan prediksi menggunakan model yang sudah dilatih
prediksi = model.predict(df)

: 

In [ ]:
print('Hasil Prediksi:', prediksi)

: 